<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=1b0f40aefb56a3203bd4f130bb4e08f52fc8a957a47794e79dac1567d2442e0e
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-10 15:17:59
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  11.04 L
Current:  1.41 C
-------------------
Today PnL: 88.67 K (0.63%)
Current PnL: -17.93 L (-12.06%)
CY Booked + Current PnL: -6.90 L (-4.64%)
-------------------
Total profit:  2.02 L
Total loss:  -19.95 L
-------------------
Total Booked + Current PnL: 20.08 L (16.5%)
Total Booked PnL: 38.01 L (31.23%)
Curr Year Booked PnL: 11.03 L (7.81%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.19 C
Est FTT PnL: 77.58 L (54.9%)
Deployed:  1.22 C
Current:  1.41 C
CAGR/XIRR %: 9.36%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,APOLLOHOSP,8285.00,-12.64,63.0,H-LC,11.21,174460.0,18283.0,7816.0,1.34,11.71,4.48,16.71,22.0,2.34,1.34,31.19,X40N,BTT,HEALTHCARE
62,SBIN,863.00,-12.30,57.0,M-LC,4.28,216889.0,15410.0,11799.0,1.19,7.65,5.44,13.51,60.0,1.31,1.67,21.21,XY25,NTT,BANKS
77,TTKPRESTIG,770.00,105.98,63.0,M-SC,5.57,88707.0,-12070.0,12162.0,-0.23,-11.98,13.71,0.09,245.0,-0.99,0.68,16.54,OX40N,NTT,DURABLES
41,ITC,452.00,-37.50,55.0,H-LC,3.25,202153.0,2015.0,18881.0,0.82,1.01,9.34,10.44,4.0,0.11,1.55,6.63,X40,NTT,FMCG
49,NATIONALUM,247.44,-44.49,67.0,H-MC,2.84,110998.0,10494.0,20146.0,1.36,10.44,18.15,30.49,79.0,0.52,0.85,46.23,MH,ATH,METALS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,SAIL,228.00,41.94,61.0,M-MC,7.06,223584.0,-1378.0,169030.0,0.30,-0.61,75.60,74.53,192.0,-0.01,1.72,31.44,XY24,BTT,STEEL
4,ANGELONE,3033.00,8.16,40.0,H-SC,6.49,191552.0,546.0,57159.0,-0.21,0.29,29.84,30.21,157.0,0.01,1.47,21.23,X40N,NTT,FINANCE
35,ICICIPRULI,790.00,-20.88,42.0,H-MC,2.00,136165.0,377.0,42375.0,0.73,0.28,31.12,31.48,107.0,0.01,1.05,12.43,X40,ATH,INSURANCE
30,HAVELLS,2069.16,-0.79,58.0,H-MC,1.76,245996.0,-1735.0,76800.0,-0.64,-0.70,31.22,30.30,92.0,-0.02,1.89,12.50,X40,ATH,ELECTRICAL
34,ICICIGI,2260.25,-20.11,44.0,H-MC,3.49,137270.0,1137.0,29993.0,1.10,0.83,21.85,22.86,91.0,0.04,1.05,13.55,X40,ATH,INSURANCE


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-15.68,40.0,H-LC,6.38,303879.0,-11963.0,124621.0,0.38,-3.79,41.01,35.67,5.0,-0.10,2.33,9.54,X40N,ATH,FMCG
43,JIOFIN,387.00,-1.80,42.0,H-LC,13.76,217140.0,-266.0,53764.0,0.73,-0.12,24.76,24.61,48.0,-0.00,1.67,54.63,XY24,BTT,FINANCE
80,UNITDSPR,1644.00,-14.51,45.0,H-LC,8.27,172036.0,-9723.0,44970.0,-0.08,-5.35,26.14,19.39,86.0,-0.22,1.32,2.36,X40N,NTT,BREWERIES
56,RELIANCE,1533.00,-14.04,46.0,H-LC,6.49,214796.0,4430.0,24358.0,0.05,2.11,11.34,13.68,37.0,0.18,1.65,18.98,XY25,NTT,REFINERIES
31,HCLTECH,1943.91,0.65,48.0,H-LC,10.16,229911.0,-12013.0,75273.0,2.61,-4.97,32.74,26.15,8.0,-0.16,1.77,8.65,X40,ATH,IT


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,LTIM,7230.20,-3.76,59.0,H-LC,1.33,238838.0,-7313.0,86531.0,2.35,-2.97,36.23,32.18,16.0,-0.08,1.83,32.00,X200,ATH,IT
50,NESTLEIND,1377.00,-7.33,62.0,H-LC,2.36,285012.0,19586.0,37194.0,1.31,7.38,13.05,21.40,11.0,0.53,2.19,14.51,XY25,NTT,FMCG
41,ITC,452.00,-37.50,55.0,H-LC,3.25,202153.0,2015.0,18881.0,0.82,1.01,9.34,10.44,4.0,0.11,1.55,6.63,X40,NTT,FMCG
20,CIPLA,1795.00,-19.78,53.0,H-LC,5.95,214350.0,9850.0,31574.0,0.84,4.82,14.73,20.25,10.0,0.31,1.65,13.97,X40N,BTT,PHARMA
83,VBL,671.64,-15.68,40.0,H-LC,6.38,303879.0,-11963.0,124621.0,0.38,-3.79,41.01,35.67,5.0,-0.10,2.33,9.54,X40N,ATH,FMCG


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAELXSI,9161.00,-17.83,62.0,H-MC,8.19,81872.0,-20780.0,46380.0,5.30,-20.24,56.65,24.94,98.0,-0.45,0.63,24.45,OX40N,NTT,IT
12,BANKINDIA,190.00,-30.43,62.0,H-MC,5.53,180623.0,815.0,111589.0,4.14,0.45,61.78,62.52,88.0,0.01,1.39,34.69,XR,NTT,BANKS
79,UNIONBANK,163.00,-16.28,53.0,M-LC,8.96,150618.0,9778.0,34717.0,4.11,6.94,23.05,31.59,66.0,0.28,1.16,35.27,XY24,NTT,BANKS
54,RAJOOENG,143.33,-34.52,39.0,H-SC,10.70,99110.0,-3390.0,44223.0,3.60,-3.31,44.62,39.83,114.0,-0.08,0.76,3.60,AR,ATH,MISC
76,TRIDENT,48.00,-7.75,57.0,M-SC,6.03,71852.0,-20402.0,46711.0,3.49,-22.12,65.01,28.51,224.0,-0.44,0.55,22.95,XR,NTT,TEXTILES


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VALIANTORG,838.00,-331.44,59.0,H-SC,8.70,98280.0,-35006.0,119597.0,-3.00,-26.26,121.69,63.47,139.0,-0.29,0.75,64.74,XR,NTT,CHEMICALS
46,LAOPALA,464.00,115.96,52.0,H-SC,3.07,68725.0,-32121.0,57949.0,-2.07,-31.85,84.32,25.61,142.0,-0.55,0.53,27.42,AR,NTT,CERAMICS
44,KANSAINER,340.00,-68.16,54.0,H-SC,2.49,219573.0,-50094.0,86424.0,-1.89,-18.58,39.36,13.47,138.0,-0.58,1.69,11.06,XY24,NTT,PAINTS
8,ATULAUTO,844.00,3667.57,61.0,M-SC,7.62,117422.0,-29946.0,81761.0,-1.40,-20.32,69.63,35.16,236.0,-0.37,0.90,19.93,XY24,NTT,AUTO
65,SIEMENS,7969.85,-2.18,51.0,H-LC,29.42,157275.0,-28820.0,241213.0,-1.40,-15.49,153.37,114.13,15.0,-0.12,1.21,15.82,AR,ATH,ELECTRICAL


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,106.53,71.0,H-SC,12.83,130707.0,-11772.0,31291.0,1.98,-8.26,23.94,13.70,163.0,-0.38,1.00,54.27,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,105.98,63.0,M-SC,5.57,88707.0,-12070.0,12162.0,-0.23,-11.98,13.71,0.09,245.0,-0.99,0.68,16.54,OX40N,NTT,DURABLES
44,KANSAINER,340.0,-68.16,54.0,H-SC,2.49,219573.0,-50094.0,86424.0,-1.89,-18.58,39.36,13.47,138.0,-0.58,1.69,11.06,XY24,NTT,PAINTS
37,INDIGOPNTS,1408.0,114.84,50.0,M-SC,3.01,140951.0,-33608.0,33645.0,0.80,-19.25,23.87,0.02,221.0,-1.00,1.08,22.60,OX40N,NTT,PAINTS
72,TATAELXSI,9161.0,-17.83,62.0,H-MC,8.19,81872.0,-20780.0,46380.0,5.30,-20.24,56.65,24.94,98.0,-0.45,0.63,24.45,OX40N,NTT,IT


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4998.00,-26.76,52.0,H-LC,13.94,292331.0,-53625.0,177474.0,1.98,-15.50,60.71,35.80,1.0,-0.30,2.24,3.56,X40,BTT,IT
39,INFY,2275.00,-16.01,58.0,H-LC,10.11,326529.0,13641.0,158040.0,1.91,4.36,48.40,54.87,3.0,0.09,2.51,11.30,X40,BTT,IT
41,ITC,452.00,-37.50,55.0,H-LC,3.25,202153.0,2015.0,18881.0,0.82,1.01,9.34,10.44,4.0,0.11,1.55,6.63,X40,NTT,FMCG
83,VBL,671.64,-15.68,40.0,H-LC,6.38,303879.0,-11963.0,124621.0,0.38,-3.79,41.01,35.67,5.0,-0.10,2.33,9.54,X40N,ATH,FMCG
1,ABB,7934.00,-40.74,48.0,H-LC,12.54,247699.0,-13920.0,133138.0,0.56,-5.32,53.75,45.57,7.0,-0.10,1.90,5.40,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,BLUESTARCO,2326.38,9.48,53.0,H-MC,3.54,189030.0,24360.0,43609.0,0.17,14.79,23.07,41.28,89.0,0.56,1.45,23.96,X40N,ATH,AC
4,ANGELONE,3033.00,8.16,40.0,H-SC,6.49,191552.0,546.0,57159.0,-0.21,0.29,29.84,30.21,157.0,0.01,1.47,21.23,X40N,NTT,FINANCE
84,VOLTAS,1918.49,-0.24,55.0,H-MC,3.32,209190.0,17448.0,78593.0,-1.25,9.10,37.57,50.08,99.0,0.22,1.61,16.52,XY25,ATH,AC
24,DABUR,735.00,-2.40,62.0,H-MC,5.15,212258.0,15230.0,74396.0,-0.36,7.73,35.05,45.49,102.0,0.20,1.63,21.78,XY24,BTT,FMCG
18,CAMS,5211.76,-4.05,54.0,H-SC,4.92,108970.0,6966.0,36963.0,1.02,6.83,33.92,43.06,122.0,0.19,0.84,25.11,X40N,ATH,MISC


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,NATIONALUM,247.44,-44.49,67.0,H-MC,2.84,110998.0,10494.0,20146.0,1.36,10.44,18.15,30.49,79.0,0.52,0.85,46.23,MH,ATH,METALS
85,WHIRLPOOL,2270.00,-39.15,63.0,M-SC,2.47,102788.0,11290.0,67460.0,-0.05,12.34,65.63,86.07,223.0,0.17,0.79,49.27,XR,NTT,DURABLES
36,INDIAMART,4850.92,-51.09,49.0,H-SC,12.14,136586.0,13250.0,120510.0,0.09,10.74,88.23,108.45,119.0,0.11,1.05,35.50,AR,ATH,MISC
86,WIPRO,420.00,-9.71,60.0,M-LC,6.49,158528.0,7583.0,101870.0,2.63,5.02,64.26,72.51,53.0,0.07,1.22,11.35,XR,NTT,IT
12,BANKINDIA,190.00,-30.43,62.0,H-MC,5.53,180623.0,815.0,111589.0,4.14,0.45,61.78,62.52,88.0,0.01,1.39,34.69,XR,NTT,BANKS


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ALKYLAMINE,4546.37,-4.17,47.0,H-SC,14.69,89139.0,-11824.0,106352.0,2.13,-11.71,119.31,93.63,148.0,-0.11,0.68,32.77,SR,ATH,CHEMICALS
22,COFFEEDAY,80.00,-39.64,58.0,L-SC,42.08,89088.0,-24461.0,64509.0,-0.62,-21.54,72.41,35.27,268.0,-0.38,0.68,118.05,XR,NTT,HOTELS
48,MASFIN,398.61,-16.44,57.0,H-SC,13.01,94950.0,-3030.0,24630.0,-0.81,-3.09,25.94,22.05,152.0,-0.12,0.73,38.02,XR,ATH,FINANCE
82,VALIANTORG,838.00,-331.44,59.0,H-SC,8.70,98280.0,-35006.0,119597.0,-3.00,-26.26,121.69,63.47,139.0,-0.29,0.75,64.74,XR,NTT,CHEMICALS
49,NATIONALUM,247.44,-44.49,67.0,H-MC,2.84,110998.0,10494.0,20146.0,1.36,10.44,18.15,30.49,79.0,0.52,0.85,46.23,MH,ATH,METALS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.0,-39.64,58.0,L-SC,42.08,89088.0,-24461.0,64509.0,-0.62,-21.54,72.41,35.27,268.0,-0.38,0.68,118.05,XR,NTT,HOTELS
60,SAMMAANCAP,326.0,-171.59,64.0,M-SC,3.95,82020.0,-20190.0,113581.0,0.28,-19.75,138.48,91.37,208.0,-0.18,0.63,33.15,XY25,NTT,FINANCE
75,TITAGARH,1548.0,-2.98,65.0,H-SC,7.10,159180.0,-32835.0,111713.0,2.90,-17.10,70.18,41.08,158.0,-0.29,1.22,34.89,XY24,NTT,ENGINEERING
8,ATULAUTO,844.0,3667.57,61.0,M-SC,7.62,117422.0,-29946.0,81761.0,-1.40,-20.32,69.63,35.16,236.0,-0.37,0.90,19.93,XY24,NTT,AUTO
64,SHALBY,327.0,1086.60,75.0,M-SC,2.33,161393.0,-19750.0,65219.0,0.66,-10.90,40.41,25.10,235.0,-0.30,1.24,29.34,XY24,NTT,HEALTHCARE


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.36
1,25,44.46
2,50,75.16


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    40.85
LC    35.47
MC    23.74
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.68
X40      16.10
X40N     13.12
XY25     10.30
XR        9.54
AR        8.36
OX40N     7.78
X200      1.83
MH        1.78
X5K       1.45
SR        1.12
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    26.88
H-SC    24.99
H-MC    20.65
M-SC    14.20
M-LC     7.48
M-MC     2.76
L-SC     1.66
L-LC     1.11
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,13.01,-15.09,73.36
FMCG,12.29,-3.62,38.75
FINANCE,9.06,-18.41,61.00
BANKS,7.81,-12.34,60.04
PAINTS,6.16,-15.23,41.02
MISC,6.06,-16.60,83.44
ELECTRICAL,5.83,-8.38,68.11
AC,3.81,2.29,30.81
AUTO,3.49,-15.74,67.94


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2733943.0,22
AR,1183456.0,9
XR,1051144.0,13
X40,870923.0,10
X40N,565897.0,10
OX40N,445716.0,10
XY25,410512.0,7
SR,197109.0,2
X5K,129400.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2768675.0,28
M-SC,1398049.0,17
H-LC,1365111.0,15
H-MC,1155601.0,15
M-LC,406977.0,5
M-MC,319259.0,2
L-SC,244937.0,3
L-MC,59797.0,1
L-LC,39204.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,917533.0,6
M-SC,XY24,815369.0,7
H-SC,AR,649361.0,5
H-LC,X40,579932.0,5
H-SC,XR,502860.0,6
H-MC,XY24,464110.0,4
H-LC,AR,374351.0,2
M-MC,XY24,319259.0,2
H-SC,X40N,247846.0,4


In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 56.0 seconds
